In [1]:
!pip install pyspark==3.1.2 py4j==0.10.9 

     |████████████████████████████████| 212.4 MB 64 kB/s 
     |████████████████████████████████| 198 kB 55.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=7cb7cf5c1a99e3983c53100130250f988b8afbd3a3db694becb1f5f85f806460
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.ml.stat import Correlation
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .master("local[4]") \
    .config("spark.driver.maxResultSize", "8g") \
    .getOrCreate()

In [3]:
from google.colab import drive               
drive.mount('/content/drive')

Mounted at /content/drive


# CAMBIARE DATASET

In [29]:
df = spark.read.json("drive/My Drive/spot_data.json")

In [2]:
df = spark.read.parquet("../data/cleanedDataset_parquet/*")

In [30]:
df.show(5)

+------------+------------------+------------+-----------+------+--------------------+--------------------+----------------+---+--------+--------+----+----------------+-----------+--------------------+---------------------+-------+--------------+-------+
|acousticness|               age|danceability|duration_ms|energy|              genres|            id_track|instrumentalness|key|liveness|loudness|mode|popularity_track|speechiness|sum_artist_followers|sum_artist_popularity|  tempo|time_signature|valence|
+------------+------------------+------------+-----------+------+--------------------+--------------------+----------------+---+--------+--------+----+----------------+-----------+--------------------+---------------------+-------+--------------+-------+
|       0.658|41.794520547945204|       0.602|     156067| 0.552|[classic czech po...|00AeAaSNbe92PRrst...|             0.0|  0|  0.0972|  -6.667|   1|               3|      0.404|               10807|                   80|182.229|    

# Preprocessing

In [3]:
from pyspark.ml.feature import QuantileDiscretizer 

qds = QuantileDiscretizer(relativeError=0.0001, handleInvalid="error", numBuckets=10, inputCol="popularity_track", outputCol="label")

df = qds.setHandleInvalid("keep").fit(df).transform(df)

In [4]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(      #con il nuovo dataset aggiungere nuovi attributi/ togliere quelli eliminati per alta correlazione
inputCols=[
 'duration_ms',
 'danceability',
 'energy',
 'key',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature',
 'avg_artist_followers',
 'avg_artist_popularity',  
 'sum_artist_followers',
 'sum_artist_popularity'], 
outputCol="feat")
df=assembler.transform(df)

In [5]:
from pyspark.ml.feature import StandardScaler


scaler = StandardScaler(inputCol="feat", outputCol="scaledFeatures")


scalerModel = scaler.fit(df)


df = scalerModel.transform(df)

In [6]:
from pyspark.ml.feature import VarianceThresholdSelector

selector = VarianceThresholdSelector(varianceThreshold=0.1,featuresCol='scaledFeatures', outputCol="features")

df = selector.fit(df).transform(df)

print("Output: Features with variance lower than %f are removed." %
      selector.getVarianceThreshold())

Output: Features with variance lower than 0.100000 are removed.


In [7]:
df.select("features").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                                           |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[1.2865419343502538,3.6454044806081134,2.1987048306672228,0.0,2.10930

In [8]:
df.show(5)

+--------------------+----------------+-----------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+--------------------+---------------------+--------------------+---------------------+------------------+-----+--------------------+--------------------+--------------------+
|            id_track|popularity_track|duration_ms|              genres|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|sum_artist_followers|sum_artist_popularity|avg_artist_followers|avg_artist_popularity|               age|label|                feat|      scaledFeatures|            features|
+--------------------+----------------+-----------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+--------------------+--------------

# Scegliere bene treshold e printare le features rimaste (se metto 1 come pasquale mi elimina tutto perchè hanno tutte varianza minore di 1), capire quali sono quelle rimaste

In [9]:
final_data = df.select("id_track", "features", "label")

In [10]:
train, test = final_data.randomSplit([0.7, 0.3], seed = 10)

In [11]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Naive Bayes

In [12]:
from pyspark.ml.classification import NaiveBayes

In [13]:
nb = NaiveBayes()                               # bisogna togliere i valori negativi, credo siano tutti in loudness, come si fa a toglierli senza rifare tutto da capo a parte solo per il nb?
nbModel = nb.fit(train)
predictions = nbModel.transform(test)

In [14]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

0.2210574055015786

In [15]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="f1")
evaluator.evaluate(predictions)

0.20290192713475574

In [16]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="recallByLabel") #recallbylabel o weightedrecall?
evaluator.evaluate(predictions)

0.5532107131332687

In [17]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="precisionByLabel") #precisionbylabel o weightedprecision?
evaluator.evaluate(predictions)

0.42052590266875983

# Random forest

In [18]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol="features",labelCol="label",maxDepth = 10)
rfModel = rf.fit(train)
predictions = rfModel.transform(test)

In [19]:
rfModel.featureImportances

SparseVector(16, {0: 0.0617, 1: 0.0215, 2: 0.0548, 3: 0.0018, 4: 0.0006, 5: 0.0436, 6: 0.1528, 7: 0.0393, 8: 0.0099, 9: 0.0134, 10: 0.0048, 11: 0.0011, 12: 0.1168, 13: 0.237, 14: 0.1075, 15: 0.1335})

In [20]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

0.29649915400582605

In [21]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="f1")
evaluator.evaluate(predictions)

0.255844560316314

In [22]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="recallByLabel") 
evaluator.evaluate(predictions)

0.7024846724749919

In [23]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="precisionByLabel") 
evaluator.evaluate(predictions)

0.6155281610495363

# MLP

In [24]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [31]:
layers = [16,3,10] # c'è un modo per scegliere i layers? questi sono scelti a caso

In [32]:
mlp = MultilayerPerceptronClassifier(labelCol='label',
                                            featuresCol='features',
                                            maxIter=100,
                                            layers=layers,
                                            blockSize=128,
                                            seed=1234)

mlpModel = mlp.fit(train)


predictions = mlpModel.transform(test)

In [33]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

Py4JJavaError: An error occurred while calling o886.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 492.0 failed 1 times, most recent failure: Lost task 1.0 in stage 492.0 (TID 2693) (192.168.56.1 executor driver): org.apache.spark.SparkException: Failed to execute user defined function(ProbabilisticClassificationModel$$Lambda$3966/915315286: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:755)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="f1")
evaluator.evaluate(predictions)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="recallByLabel") 
evaluator.evaluate(predictions)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="precisionByLabel") 
evaluator.evaluate(predictions)

# LSVM one vs others

In [34]:
from pyspark.ml.classification import LinearSVC, OneVsRest

In [35]:
lsvc = LinearSVC(maxIter=10, regParam=0.1)

In [36]:
ovr = OneVsRest(classifier=lsvc)

In [ ]:
ovrModel = ovr.fit(train)

In [ ]:
predictions = ovrModel.transform(test)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="f1")
evaluator.evaluate(predictions)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="recallByLabel") 
evaluator.evaluate(predictions)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="precisionByLabel") 
evaluator.evaluate(predictions)

# Logistic regression one vs others

In [60]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, tol=1E-6, fitIntercept=True) # i parametri vanno bene?

In [61]:
ovr = OneVsRest(classifier=lr)

In [62]:
ovrModel = ovr.fit(train)

In [63]:
predictions = ovrModel.transform(test)

In [64]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

0.23162846390694491

In [65]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="f1")
evaluator.evaluate(predictions)

0.18375327687152626

In [66]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="recallByLabel") 
evaluator.evaluate(predictions)

0.6367624083643793

In [67]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="recallByLabel") 
evaluator.evaluate(predictions)

0.6367624083643793